# Qiskit Runtime API

Qiskit Runtime is a cloud service that allow you upload and run Qiskit programs in a Qiskit Runtimes near to the QPUs. 

In this example we are going to explore how to integrate this Qiskit Runtime in any application using only basic HTTP requests and interact with a the program running in the service.



<div class="alert alert-block alert-info">
<b>Note:</b> You need to have an IBM Quantum API token. You can get one by signing up at https://quantum-computing.ibm.com/
</div>

For this tutorial we are going to use the Python HTTP library `requests` to facilitate the API calls.

You can access the online interactive runtime API documentation here: https://runtime-us-east.quantum-computing.ibm.com/openapi/

In [70]:
import requests
import json
import time

## Authenticate with the service 

In [51]:
# you need to pass your API token in the header of every call

headers = {
  'Authorization': f'Bearer {Your_API_TOKEN}',
  'Content-Type': 'application/json'
}

Runtime_API_URL = "https://runtime-us-east.quantum-computing.ibm.com/"

## Get a list of program that you can invoke 

In [52]:

# Call the API passing the API Token and get the programs

response = requests.get(Runtime_API_URL + 'programs' , headers = headers)

list_of_programs = {}
if response.status_code == 200:
    list_of_programs = response.json()
else:
    print(f'Error:{response.status_code}')
    exit()
    
print(f'Qiskit Runtime Programs:')

for program in list_of_programs["programs"]:
    print(f'- {program["name"]}: {program["description"]} ')


Qiskit Runtime Programs:
- qasm3-runner: A runtime program that takes one or more circuits, converts them to OpenQASM3, compiles them, executes them, and optionally applies measurement error mitigation. This program can also take and execute one or more OpenQASM3 strings. Note that this program can only run on a backend that supports OpenQASM3. 
- sampler: Sample distributions generated by given circuits executed on the target backend. 
- estimator: Expectation value estimator. A runtime program that estimates the value of an observable for an input quantum circuit. This program is in beta mode and is only available to select accounts. 
- sample-expval: A sample expectation value program. 
- vqe: Variational Quantum Eigensolver (VQE) to find the minimal eigenvalue of a Hamiltonian. 
- circuit-runner: A runtime program that takes one or more circuits, compiles them, executes them, and optionally applies measurement error mitigation. 
- hello-world: A sample runtime program. 
- quantum-k

## Display information about a specific program

In [53]:
print('Qiskit Program details:\n')

qiskit_runtime_program = json.dumps(list_of_programs["programs"][0], indent=2) 
                                    
print(qiskit_runtime_program)

Qiskit Program details:

{
  "id": "qasm3-runner",
  "name": "qasm3-runner",
  "cost": 14400,
  "description": "A runtime program that takes one or more circuits, converts them to OpenQASM3, compiles them, executes them, and optionally applies measurement error mitigation. This program can also take and execute one or more OpenQASM3 strings. Note that this program can only run on a backend that supports OpenQASM3.",
  "creation_date": "2021-11-08T18:28:28Z",
  "update_date": "2021-11-10T17:10:19.353876Z",
  "is_public": true,
  "spec": {
    "backend_requirements": {
      "input_allowed": "qasm3"
    },
    "parameters": {
      "$schema": "https://json-schema.org/draft/2019-09/schema",
      "properties": {
        "circuits": {
          "description": "A circuit/OpenQASM3 string or a list of circuits/OpenQASM3 strings (real backend only).",
          "type": [
            "object",
            "array",
            "string"
          ]
        },
        "exporter_config": {
       

## Run a Qiskit Runtime program on the Cloud

In [83]:
Runtime_program = "hello-world"

program_input = {"iterations":2}   

def run_runtime_program(program_name, program_input):
    # configuing your IBM Provider data
    params = json.dumps({
      "program_id": Runtime_program,
      "hub": "ibm-q-internal",
      "group": "near-time",
      "project": "qiskit-runtime",
      "backend": "ibmq_montreal",
      "params": program_input
    })

    job_ID = ''

    response = requests.post(Runtime_API_URL + 'jobs', data=params, headers=headers)

    if response.status_code == 200:
        job_ID =  response.json()['id']
        return 200, job_ID
    else:
        return response.status_code, None

    
status, job_ID = run_runtime_program(Runtime_program, program_input)

print(f' status: {status}, Job: {job_ID}')

    

 status: 200, Job: c2b04fot418h0qp0peh0


## Get the result

In [81]:
# You are going to get a 204 status code while the Qiskit program is still running.

final_result = False

while not final_result:
    
    response = requests.get(Runtime_API_URL + 'jobs/'+ job_ID +'/results', headers=headers)

    if response.status_code == 200:
        print(f'Final Result: {response.text}')
        final_result = True
        
    elif response.status_code == 204:
        print(f'Waiting for the final result')
        time.sleep(2)
        
    else:
        print(f'Error:{response.status_code}')
        break

Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Waiting for the final result
Final Result: All done!
